# Étape 3 — Indicateurs Clés de Performance (ICP)

L’objectif de cette étape est de calculer et analyser les principaux indicateurs
de performance commerciale et de satisfaction client à partir des données
nettoyées issues de l’étape précédente.

Les indicateurs sont construits en privilégiant des mesures économiquement
pertinentes, notamment des taux calculés en valeur lorsque cela est justifié.


**Import des librairies**

In [ ]:
# Import des librairies nécessaires à l'analyse des ICP

import pandas as pd
import numpy as np

# Options d'affichage pour une meilleure lisibilité
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", "{:,.2f}".format)

**Chargement des données nettoyées**

In [ ]:
# Chargement des données nettoyées depuis le dossier data/processed

order_lines = pd.read_csv("../data/processed/order_lines_clean.csv")
customers = pd.read_csv("../data/processed/customers_clean.csv")
products = pd.read_csv("../data/processed/products_clean.csv")

**Préparation des données pour l’analyse ICP**

In [ ]:
# Conversion des variables de date
order_lines["order_date"] = pd.to_datetime(order_lines["order_date"])

# Création de la variable mois de commande
order_lines["order_month"] = order_lines["order_date"].dt.to_period("M")

# Vérification rapide de la structure des données
order_lines[[
    "order_id",
    "order_date",
    "order_month",
    "net_amount",
    "quantity",
    "returned",
    "category",
    "channel",
    "review_score",
    "delivery_days"
]].head()


**Chiffre d’affaires total (CA)**

In [ ]:
# Calcul du chiffre d'affaires total (CA)
# Le CA est calculé à partir du montant net (net_amount),
# c’est-à-dire après application des remises.

ca_total = order_lines["net_amount"].sum()

# Affichage du résultat
print(f"Chiffre d'affaires total (CA net) : {ca_total:,.2f}")

**Chiffre d’affaires mensuel**

In [ ]:
# Calcul du chiffre d'affaires mensuel à partir du CA net
# Agrégation par mois de commande (order_month)

ca_mensuel = (
    order_lines
    .groupby("order_month", as_index=False)
    .agg(CA_mensuel=("net_amount", "sum"))
    .sort_values("order_month")
)

# Affichage du chiffre d'affaires mensuel
ca_mensuel


**Export du chiffre d'affaires mesnsuel vers results/tables**

In [ ]:
ca_mensuel.to_csv("../results/tables/ca_mensuel.csv", index=False)

**chiffre d'affaires net par catégorie et par canal**

In [ ]:
# Tableau croisé : chiffre d'affaires net par catégorie et par canal
# Utilisation explicite de pivot_table (analyse multidimensionnelle)

ca_pivot = pd.pivot_table(
    order_lines,
    values="net_amount",
    index="category",
    columns="channel",
    aggfunc="sum",
    margins=True
)

ca_pivot

**Export du chiffre d'affaires net par catégorie et par canal vers results/tables**

In [ ]:
ca_pivot.to_csv("../results/tables/ca_pivot.csv", index=False)

**Panier moyen global (AOV)**

In [ ]:
# Calcul du panier moyen global (Average Order Value - AOV)
# AOV = Chiffre d'affaires total / Nombre de commandes distinctes

# Nombre total de commandes
nb_commandes = order_lines["order_id"].nunique()

# Panier moyen global
aov_global = ca_total / nb_commandes

# Affichage du résultat
print(f"Panier moyen global (AOV) : {aov_global:,.2f}")


**Panier moyen mensuel (AOV mensuel)**

In [ ]:
# Calcul du panier moyen mensuel (AOV mensuel)
# AOV mensuel = CA mensuel / Nombre de commandes mensuelles

aov_mensuel = (
    order_lines
    .groupby("order_month")
    .agg(
        CA_mensuel=("net_amount", "sum"),
        nb_commandes=("order_id", "nunique")
    )
    .assign(AOV_mensuel=lambda df: df["CA_mensuel"] / df["nb_commandes"])
    .reset_index()
    .sort_values("order_month")
)

# Affichage du panier moyen mensuel
aov_mensuel

**Export du panier moyen mesnsuel vers results/tables**

In [ ]:
aov_mensuel.to_csv("../results/tables/aov_mensuel.csv" , index=False)

## Méthodologie de calcul du taux de remise

Les taux de remise peuvent être calculés soit en volume (nombre d’articles),
soit en valeur monétaire. Dans le cadre de ce projet, le calcul est effectué
**en valeur**, afin de tenir compte de l’hétérogénéité des prix entre les
produits et les catégories.

Le calcul en volume attribuerait le même poids à des articles de faible valeur
et à des articles de valeur élevée, ce qui pourrait biaiser l’analyse
économique. À l’inverse, le calcul en valeur permet de mesurer l’impact réel
des remises sur le chiffre d’affaires.

**Taux de remise moyen global (en valeur)**

In [ ]:
# Calcul du taux de remise moyen global (en valeur)

# Calcul du prix catalogue (avant remise)
order_lines["catalog_amount"] = order_lines["unit_price"] * order_lines["quantity"]

# Calcul du montant total des remises
total_discount_value = (order_lines["catalog_amount"] - order_lines["net_amount"]).sum()

# Calcul du montant total catalogue
total_catalog_value = order_lines["catalog_amount"].sum()

# Taux de remise moyen global (en valeur)
taux_remise_global = total_discount_value / total_catalog_value

# Affichage du résultat
print(f"Taux de remise moyen global (en valeur) : {taux_remise_global:.2%}")

**Taux de remise moyen par catégorie (en valeur)**

In [ ]:
# Calcul du taux de remise moyen par catégorie (en valeur)

taux_remise_par_categorie = (
    order_lines
    .groupby("category")
    .agg(
        discount_value=("catalog_amount", 
                        lambda x: (x - order_lines.loc[x.index, "net_amount"]).sum()),
        catalog_value=("catalog_amount", "sum")
    )
    .assign(taux_remise=lambda df: df["discount_value"] / df["catalog_value"])
    .sort_values("taux_remise", ascending=False)
    .reset_index()
)

# Affichage du taux de remise par catégorie
taux_remise_par_categorie

**Export du taux de remise par catégorie vers results/tables**

In [ ]:
taux_remise_par_categorie.to_csv("../results/tables/taux_remise_par_categorie.csv", index=False)

**Top 5 catégories les plus remisées**

In [ ]:
# Top 5 catégories avec le taux de remise le plus élevé

top_5_remises = taux_remise_par_categorie.nlargest(5, "taux_remise")

top_5_remises

**Export du top_5 vers results/tables**

In [ ]:
top_5_remises.to_csv("../results/tables/top_5_remises", index=False)

## Méthodologie de calcul des taux de retour

Les taux de retour peuvent être calculés soit en volume (nombre d’articles ou de
commandes retournées), soit en valeur monétaire. Dans le cadre de ce projet,
l’analyse privilégie les **taux de retour calculés en valeur**, considérés comme
les plus pertinents d’un point de vue économique.

En effet, le calcul en volume attribue le même poids à toutes les unités
vendues, indépendamment de leur prix. Or, le jeu de données étudié présente une
forte hétérogénéité des montants de vente selon les produits, les catégories et
les canaux de distribution.

Le calcul en valeur permet de mesurer l’impact réel des retours sur le chiffre
d’affaires, en rapportant le montant des ventes retournées au montant total des
ventes. Cette approche est particulièrement adaptée pour l’aide à la décision
économique et commerciale.

**Taux de retour global**

In [ ]:
# Calcul du taux de retour global

# --- Taux de retour en volume ---
# Nombre total d'articles vendus
articles_vendus = order_lines["quantity"].sum()

# Nombre total d'articles retournés
articles_retournes = order_lines.loc[order_lines["returned"] == 1, "quantity"].sum()

taux_retour_volume = articles_retournes / articles_vendus


# --- Taux de retour en valeur ---
# Montant total des ventes
valeur_ventes_totales = order_lines["net_amount"].sum()

# Montant des ventes retournées
valeur_ventes_retournees = order_lines.loc[
    order_lines["returned"] == 1, "net_amount"
].sum()

taux_retour_valeur = valeur_ventes_retournees / valeur_ventes_totales


# Affichage des résultats
print(f"Taux de retour global (en volume) : {taux_retour_volume:.2%}")
print(f"Taux de retour global (en valeur) : {taux_retour_valeur:.2%}")

**Taux de retour par catégorie (en valeur)**

In [ ]:
# Calcul du taux de retour par catégorie (en valeur)

taux_retour_par_categorie = (
    order_lines
    .groupby("category")
    .agg(
        ventes_totales=("net_amount", "sum"),
        ventes_retournees=("net_amount", 
                            lambda x: x[order_lines.loc[x.index, "returned"] == 1].sum())
    )
    .assign(taux_retour=lambda df: df["ventes_retournees"] / df["ventes_totales"])
    .sort_values("taux_retour", ascending=False)
    .reset_index()
)

# Affichage du taux de retour par catégorie
taux_retour_par_categorie

**Export du taux de retour par catégorie vers results/tables**

In [ ]:
taux_retour_par_categorie.to_csv("../results/tables/taux_retour_par_categorie.csv", index=False)

**Taux de retour par canal (en valeur)**

In [ ]:
# Calcul du taux de retour par canal de vente (en valeur)

taux_retour_par_canal = (
    order_lines
    .groupby("channel")
    .agg(
        ventes_totales=("net_amount", "sum"),
        ventes_retournees=("net_amount", 
                            lambda x: x[order_lines.loc[x.index, "returned"] == 1].sum())
    )
    .assign(taux_retour=lambda df: df["ventes_retournees"] / df["ventes_totales"])
    .sort_values("taux_retour", ascending=False)
    .reset_index()
)

# Affichage du taux de retour par canal
taux_retour_par_canal

**Export du taux de retour par canal vers results/tables**

In [ ]:
taux_retour_par_canal.to_csv("../results/tables/taux_retour_par_canal.csv", index=False)

**Top 5 catégories avec le plus fort taux de retour (en valeur)**

In [ ]:
# Top 5 catégories avec le taux de retour le plus élevé

top_5_retours = taux_retour_par_categorie.nlargest(5, "taux_retour")

top_5_retours


**Export du taux de retours vers results/tables**

In [ ]:
top_5_retours.to_csv("../results/tables/top_5_retours", index=False)

**Score d’avis moyen global**

In [ ]:
# Calcul du score d'avis moyen global
# Moyenne simple des scores d'avis après nettoyage

score_avis_global = order_lines["review_score"].mean()

# Affichage du résultat
print(f"Score d'avis moyen global : {score_avis_global:.2f}")

**Score d’avis moyen par catégorie**

In [ ]:
# Calcul du score d'avis moyen par catégorie de produit

score_avis_par_categorie = (
    order_lines
    .groupby("category")
    .agg(score_moyen=("review_score", "mean"))
    .sort_values("score_moyen", ascending=False)
    .reset_index()
)

# Affichage du score d'avis moyen par catégorie
score_avis_par_categorie

**Export du score moyen par catégorie vers results/tables**

In [ ]:
score_avis_par_categorie.to_csv("../results/tables/score_avis_par_categorie.csv", index=False)

**Création des intervalles de délai de livraison**

In [ ]:
# Création d'intervalles numériques pour le délai de livraison (en jours)
# Intervalles retenus :
# 0–2 jours : livraison très rapide
# 3–5 jours : livraison standard
# 6–10 jours : livraison lente
# > 10 jours : livraison très tardive

bins = [0, 2, 5, 10, np.inf]
labels = ["0-2 jours", "3-5 jours", "6-10 jours", "> 10 jours"]

order_lines["delivery_delay_class"] = pd.cut(
    order_lines["delivery_days"],
    bins=bins,
    labels=labels,
    right=True,
    include_lowest=True
)

# Vérification rapide de la répartition par classe de délai
order_lines["delivery_delay_class"].value_counts().sort_index()

**Score d’avis moyen par délai de livraison**

In [ ]:
# Calcul du score d'avis moyen par intervalle de délai de livraison

score_avis_par_delai = (
    order_lines
    .groupby("delivery_delay_class")
    .agg(
        score_moyen=("review_score", "mean"),
        nb_commandes=("order_id", "nunique")
    )
    .reset_index()
)

# Affichage des résultats
score_avis_par_delai

**Export score_avis par delai vers results/tables**

In [ ]:
score_avis_par_delai.to_csv("../results/tables/score_avis_par_delai.csv", index=False)